# Chapter 12: Signaling adversity

## Determine the kinds

In [1]:
id :: a -> a
-- a is *

: 

In [2]:
r :: a -> f a
-- a is *
-- f is * -> *

: 

## String processing

In [3]:
replaceThe :: String -> String
replaceThe = unwords . fmap normalize . words where
    normalize "the" = "a"
    normalize s = s
    
replaceThe "the cow loves the us"

"a cow loves a us"

In [4]:
countTheBeforeVowel :: String -> Int
countTheBeforeVowel [] = 0
countTheBeforeVowel str = length vs where
    ws = words str
    ps = zip ws (tail ws)
    thes = filter ((== "the") . fst) ps
    vs = filter ((`elem` "eyuioa") . head . snd) thes
    
countTheBeforeVowel ""
countTheBeforeVowel "hello"
countTheBeforeVowel "hello the"
countTheBeforeVowel "hello the qwer"
countTheBeforeVowel "hello the awer the basdf the aaa"

0

0

0

0

2

In [5]:
countTheBeforeVowel :: String -> Int
countTheBeforeVowel = length . filter ((`elem` "eyuioa") . head . snd) . filter ((== "the") . fst) . zipWithRest . words where
    zipWithRest xs = zip xs (tail xs)
    
countTheBeforeVowel ""
countTheBeforeVowel "hello"
countTheBeforeVowel "hello the"
countTheBeforeVowel "hello the qwer"
countTheBeforeVowel "hello the awer the basdf the aaa"

0

0

0

0

2

In [6]:
class Truthy a where
    isTruthy :: a -> Bool
    
instance (Truthy a, Truthy b) => Truthy (a, b) where
    isTruthy (a, b)
        | isTruthy a && isTruthy b = True
        | otherwise = False
        
instance Truthy Bool where
    isTruthy x = x

instance Truthy a => Truthy (Maybe a) where
    isTruthy Nothing = False
    isTruthy (Just x) = isTruthy x
    
isTruthy True
isTruthy False
isTruthy (True, True)
isTruthy (True, False)

isTruthy (Nothing :: Maybe Bool)
isTruthy (Just True)
isTruthy (Just False)

True

False

True

False

False

True

False

In [7]:
import Data.Bifunctor (bimap)

safeHead :: [a] -> Maybe a
safeHead [] = Nothing
safeHead (x:_) = Just x

isVowel :: Char -> Bool
isVowel = (`elem` "eyuioa")

countTheBeforeVowel :: String -> Int
countTheBeforeVowel = length . filter isProperPair . zipWithRest . words where
    isVowelWord = isTruthy . fmap isVowel . safeHead
    isThe = (== "the")
    isProperPair = isTruthy . bimap isThe isVowelWord
    zipWithRest xs = zip xs (tail xs)
    
countTheBeforeVowel ""
countTheBeforeVowel "hello"
countTheBeforeVowel "hello the"
countTheBeforeVowel "hello the qwer"
countTheBeforeVowel "hello the awer the basdf the aaa"

0

0

0

0

2

In [8]:
countVowels :: String -> Int
countVowels = length . filter isVowel

countVowels "qweraa"

3

## Validate the word

In [9]:
newtype Word' = Word' String deriving (Eq, Show)

mkWord :: String -> Maybe Word'
mkWord str
    | numVowels > numConsonants = Nothing
    | otherwise = Just $ Word' str
    where
        numVowels = countVowels str
        numConsonants = length str - numVowels
        
mkWord "qwer"
mkWord "aaeass"

Just (Word' "qwer")

Nothing

## It's only natural

In [10]:
data Nat
    = Zero
    | Succ Nat
    deriving (Eq, Show)
    
natToInteger :: Nat -> Integer
natToInteger Zero = 0
natToInteger (Succ n) = 1 + natToInteger n

natToInteger Zero
natToInteger (Succ Zero)
natToInteger (Succ (Succ Zero))

0

1

2

In [11]:
integerToNat :: Integer -> Maybe Nat
integerToNat n
    | n < 0 = Nothing
    | n == 0 = Just Zero
    | Just p <- integerToNat (n - 1) = Just $ Succ p
    | otherwise = Nothing

integerToNat (-10)
integerToNat 0
integerToNat 1
integerToNat 3

Nothing

Just Zero

Just (Succ Zero)

Just (Succ (Succ (Succ Zero)))

In [12]:
integerToNat :: Integer -> Maybe Nat
integerToNat int
    | int < 0 = Nothing
    | otherwise = Just $ f int
    where
        f 0 = Zero
        f n = Succ $ f (n - 1)
        
integerToNat (-10)
integerToNat 0
integerToNat 1
integerToNat 3        

Nothing

Just Zero

Just (Succ Zero)

Just (Succ (Succ (Succ Zero)))

In [13]:
import Control.Applicative (Alternative)
import Control.Monad (guard)

ensure :: Alternative f => (a -> Bool) -> a -> f a
ensure p a = a <$ guard (p a)

(ensure (> 10) 123) :: Maybe Int
(ensure (> 10) 5) :: Maybe Int

Just 123

Nothing

In [14]:
:t (<$)

(<$) :: forall (f :: * -> *) a b. Functor f => a -> f b -> f a

In [15]:
:t guard

guard :: forall (f :: * -> *). Alternative f => Bool -> f ()

In [16]:
integerToNat :: Integer -> Maybe Nat
integerToNat = fmap f . ensure (>= 0) where
    f 0 = Zero
    f n = Succ $ f (n - 1)
        
integerToNat (-10)
integerToNat 0
integerToNat 1
integerToNat 3        

Nothing

Just Zero

Just (Succ Zero)

Just (Succ (Succ (Succ Zero)))

## Small library for Maybe

In [17]:
isJust :: Maybe a -> Bool
isJust Nothing = False
isJust (Just _) = True

isJust (Just 12)
isJust Nothing

True

False

In [18]:
isNothing :: Maybe a -> Bool
isNothing = not . isJust

isNothing (Just 12)
isNothing Nothing

False

True

In [19]:
import Data.Maybe (fromMaybe)

maybe' :: b -> (a -> b) -> Maybe a -> b
maybe' b a2b ma = fromMaybe b (a2b <$> ma)

Line 4: Use maybe
Found:
fromMaybe b (a2b <$> ma)
Why not:
maybe b a2b ma

In [20]:
listToMaybe :: [a] -> Maybe a
listToMaybe [] = Nothing
listToMaybe [a] = Just a

In [21]:
maybeToList :: Maybe a -> [a]
maybeToList Nothing = []
maybeToList (Just x) = [x]

In [22]:
catMaybes :: [Maybe a] -> [a]
catMaybes = (>>= maybeToList)

In [23]:
flipMaybe :: [Maybe a] -> Maybe [a]
flipMaybe [] = Just []
flipMaybe (Nothing:_) = Nothing
flipMaybe (Just x:xs) = (x:) <$> flipMaybe xs

In [24]:
:t foldr

foldr :: forall (t :: * -> *) a b. Foldable t => (a -> b -> b) -> b -> t a -> b

In [25]:
import Control.Applicative (liftA2)

flipMaybe :: [Maybe a] -> Maybe [a]
flipMaybe = foldr (liftA2 (:)) (Just [])

flipMaybe [Just 1, Just 2, Just 3]
flipMaybe [Just 1, Nothing, Just 3]

Just [1,2,3]

Nothing

## Small library for Either

In [26]:
left :: Either a b -> Maybe a
left (Left a) = Just a
left _ = Nothing

lefts' :: [Either a b] -> [a]
lefts' = foldr ((++) . maybeToList . left) []

Line 6: Use concatMap
Found:
foldr ((++) . maybeToList . left) []
Why not:
concatMap (maybeToList . left)

In [27]:
lefts' :: [Either a b] -> [a]
lefts' = concatMap (maybeToList . left)

Line 2: Use mapMaybe
Found:
concatMap (maybeToList . left)
Why not:
Data.Maybe.mapMaybe left

In [28]:
import Data.Maybe (mapMaybe)

lefts' :: [Either a b] -> [a]
lefts' = mapMaybe left

In [29]:
swapEither :: Either a b -> Either b a
swapEither (Left a) = Right a
swapEither (Right b) = Left b

right :: Either a b -> Maybe b
right = left . swapEither

rights' :: [Either a b] -> [b]
rights' = lefts' . fmap swapEither

In [30]:
compose2 :: (a' -> b' -> c) -> (a -> a') -> (b -> b') -> a -> b -> c
compose2 ff f g x y = ff (f x) (g y)

partitionEithers' :: [Either a b] -> ([a], [b])
partitionEithers' xs = (lefts' xs, rights' xs)

In [31]:
eitherMaybe' :: (b -> c) -> Either a b -> Maybe c
eitherMaybe' f = fmap f . right

In [32]:
either' :: (a -> c) -> (b -> c) -> Either a b -> c
either' a2c _ (Left a) = a2c a
either' _ b2c (Right b) = b2c b

In [33]:
eitherMaybe'' :: (b -> c) -> Either a b -> Maybe c
eitherMaybe'' f = either' (const Nothing) (Just . f)

## Unfolds

Nothing to do here

## Why bother?

Nothing to do here

## Write your own iterate and unfoldr

In [34]:
:t iterate

iterate :: forall a. (a -> a) -> a -> [a]

In [35]:
take 10 $ iterate (+1) 1

[1,2,3,4,5,6,7,8,9,10]

In [36]:
myIterate :: (a -> a) -> a -> [a]
myIterate f a = a:myIterate f (f a)
    
take 10 $ myIterate (+1) 1

[1,2,3,4,5,6,7,8,9,10]

In [37]:
myUnfoldr :: (b -> Maybe (a, b)) -> b -> [a]
myUnfoldr f b = case f b of
    Nothing -> []
    Just (a, b') -> a:myUnfoldr f b'

In [38]:
betterIterate :: (a -> a) -> a -> [a]
betterIterate f = myUnfoldr f' where
    f' x = Just (x, f x)
    
take 10 $ betterIterate (+1) 1

[1,2,3,4,5,6,7,8,9,10]

## Finally something other than list!

Fuck the trees